# TensorWaves demo PANDA Seminar December 2021

This notebook accompanies [these slides](https://docs.google.com/presentation/d/e/2PACX-1vSymz5AjdhPw4Kz1pKhdFMnFGYuQvVaC8WbV_HTg770x6RDYoP-Anv9tn88DSuzvSiiQ9F4pcDGVExv/pub). They were presented during a PANDA Seminar on 13 December 2021.

_This notebook is based on [`tensorwaves.ipynb`](./tensorwaves-optimized-expression.ipynb). In addition to simplifying the amplitude model expression, it illustrates how to cache constant sub-trees in that expression._

Related notebooks for this presentation:
- [QRules demo](./qrules.ipynb)
- [AmpForm demo](./ampform.ipynb)

For more extensive examples, see **[tensorwaves.rtfd.io](https://tensorwaves.readthedocs.io)**.

## Install dependencies

In [ ]:
%pip install -q ampform[viz]==0.12.* tensorwaves[jax,pwa]==0.4.*

In [ ]:
%load_ext autoreload
%autoreload
%config InlineBackend.figure_formats = ['svg']
import logging
import re
import warnings

import ampform
import graphviz
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import qrules
import sympy as sp
import tensorflow as tf
from ampform.dynamics.builder import create_relativistic_breit_wigner_with_ff
from IPython.display import Math, display
from jax.lib import xla_bridge
from matplotlib import cm
from tensorflow.python.ops.numpy_ops import np_config
from tensorwaves.data import (
    IntensityDistributionGenerator,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
    TFWeightedPhaseSpaceGenerator,
)
from tensorwaves.data.transform import SympyDataTransformer
from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.optimizer.callbacks import CSVSummary
from tensorwaves.optimizer.minuit import Minuit2

LOGGER = logging.getLogger("absl")
LOGGER.setLevel(logging.ERROR)
tf.get_logger().setLevel("WARNING")
warnings.filterwarnings("ignore")

mpl.rcParams.update({"font.size": 14})
np_config.enable_numpy_behavior()

has_tf_gpu = bool(tf.config.list_physical_devices("GPU"))
jax_backend = xla_bridge.get_backend().platform.upper()
print("TF backend: ", "GPU" if has_tf_gpu else "CPU")
print("JAX backend:", jax_backend)

Some helper functions for visualizing the distributions and fit result:

In [ ]:
def indicate_masses():
    reaction_info = model.reaction_info
    resonances = sorted(
        reaction_info.get_intermediate_particles(),
        key=lambda p: p.mass,
    )
    evenly_spaced_interval = np.linspace(0, 1, len(resonances))
    colors = [cm.rainbow(x) for x in evenly_spaced_interval]
    plt.xlabel("$m$ [GeV]")
    for i, p in enumerate(resonances):
        plt.gca().axvline(x=p.mass, linestyle="dotted", label=p.name, color=colors[i])


def compare_model(
    variable_name,
    data_set,
    phsp_set,
    intensity_model,
    bins=100,
):
    data = np.array(data_set[variable_name])
    phsp = np.array(phsp_set[variable_name])
    intensities = np.array(intensity_model(phsp_set))
    _, ax = plt.subplots(figsize=(10, 3.5))
    ax.hist(
        data,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    ax.hist(
        phsp,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="model",
        density=True,
    )
    ax.set_yticks([])
    indicate_masses()
    ax.legend()


def natural_sorting(text):
    # https://stackoverflow.com/a/5967539/13219025
    return [
        __attempt_number_cast(c)
        for c in re.split(r"[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)", text)
    ]


def __attempt_number_cast(text):
    try:
        return float(text)
    except ValueError:
        return text

## Formulate amplitude model

Generate allowed transitions for $J/\psi \to \gamma f_0, f_0 \to \pi^0 \pi^0$ with QRules (see [QRules demo](./qrules.ipynb)):

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "EM"],
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

Express the transitions as an amplitude model with the resonances parametrized as a relativistic Breit-Wigner with form factor (see [AmpForm demo](./ampform.ipynb)):

In [ ]:
builder = ampform.get_builder(reaction)
resonances = reaction.get_intermediate_particles()
for p in resonances:
    builder.set_dynamics(p.name, create_relativistic_breit_wigner_with_ff)
model = builder.formulate()

## Generate data

Generate a **deterministic** phase-space sample for this decay and an intensity-based hit-and-miss sample for this amplitude model (`intensity`):

In [ ]:
expression = model.expression.doit()
parameter_defaults = model.parameter_defaults
intensity = create_parametrized_function(expression, parameter_defaults, backend="jax")
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)
reaction_info = model.reaction_info
initial_state_mass = reaction_info.initial_state[-1].mass
final_state_masses = {i: p.mass for i, p in reaction_info.final_state.items()}
rng = TFUniformRealNumberGenerator(seed=0)

phsp_generator = TFPhaseSpaceGenerator(initial_state_mass, final_state_masses)
phsp_momenta = phsp_generator.generate(1_000_000, rng)
phsp = helicity_transformer(phsp_momenta)

data_generator = IntensityDistributionGenerator(
    function=intensity,
    domain_generator=TFWeightedPhaseSpaceGenerator(
        initial_state_mass, final_state_masses
    ),
    domain_transformer=helicity_transformer,
)
data_momenta = data_generator.generate(100_000, rng)
data = helicity_transformer(data_momenta)

In [ ]:
fig, ax = plt.subplots()
ax.hist(np.array(data["m_12"]), density=True, bins=100, alpha=0.5, label="data")
ax.hist(np.array(phsp["m_12"]), density=True, bins=100, alpha=0.5, label="phsp")
ax.set_xlabel(R"$m_{\pi^0\pi^0}$")
ax.set_yticks([])
plt.legend()
plt.show()

## Perform fit

In the following, we give a few of the parameters in the amplitude model a little offset from the original values suggested by AmpForm. These parameters will later be optimized with regard to the generated data sample. The model with offset parameters looks like this:

In [ ]:
initial_parameters = {
    "m_f(0)(500)": 0.35,
    "m_f(0)(980)": 0.88,
    "m_f(0)(1370)": 1.22,
    "m_f(0)(1500)": 1.45,
    "m_f(0)(1710)": 1.83,
    "Gamma_f(0)(500)": 0.3,
    "Gamma_f(0)(980)": 0.1,
    "Gamma_f(0)(1710)": 0.3,
}
original_parameters = intensity.parameters
intensity.update_parameters(initial_parameters)
compare_model("m_12", data, phsp, intensity, bins=200)

### Collapsing constant sub-trees

Having decided which parameters are to be optimized, we can apply some additional optimizations to the amplitude model expression, so that the computations during the fit are faster. The first of these optimizations is to substitute the parameters that remain fixed with their suggested parameter values. Note how this decreases the number of operations (nodes) in the expression tree:

In [ ]:
sp.count_ops(expression)

In [ ]:
free_parameters = {p for p in model.parameter_defaults if p.name in initial_parameters}
fixed_parameters = {
    p: v for p, v in model.parameter_defaults.items() if p not in free_parameters
}
optimized_expression = expression.subs(fixed_parameters)
sp.count_ops(optimized_expression)

In addition, we can substitute the final state masses $m_0, m_1, m_2$ with scalar values, as the final states in this decay channel are stable. SymPy will then further simply the expression, so that the computation is less complex.

In [ ]:
optimized_expression = optimized_expression.subs({
    sp.Symbol(f"m_{i}", real=True): particle.mass
    for i, particle in reaction.final_state.items()
})
sp.count_ops(optimized_expression)

Note that there are a few irrational numbers (square roots) in the expression as well, for example in this amplitude:

In [ ]:
optimized_expression.args[0].args[0].args[0].args[0]

When we substitute these values, the expression tree becomes even smaller.

In [ ]:
for node in sp.preorder_traversal(optimized_expression):
    if node.free_symbols:
        continue
    if isinstance(node, sp.Pow):
        optimized_expression = optimized_expression.xreplace({node: node.n()})
sp.count_ops(optimized_expression)

Also note that SymPy has rendered $\sqrt{m_{12}^2}$ in each Breit-Wigner as $|m_{12}|$. This can be substituted as follows:

In [ ]:
m_12 = sp.Symbol("m_12", real=True)
optimized_expression = optimized_expression.subs({sp.Abs(m_12): m_12})
sp.count_ops(optimized_expression)

Finally, note that some of the computed kinematic variable arrays are complex-valued. This can be useful in come cases, but in this decay channel, all kinematic variable values lie on the real axis. The fit can therefore be further optimized by casting the data arrays to real values:

In [ ]:
data_real = {var: array.real for var, array in data.items()}
phsp_real = {var: array.real for var, array in phsp.items()}

### Caching

In [ ]:
def cache_constant_sub_expressions(expression, free_symbols):
    cse, cached_expressions = sp.cse(expression, ignore=free_symbols)
    assert len(cached_expressions) == 1
    cached_expression = cached_expressions[0]
    cse_dict = dict(cse)
    cse_symbols = set(cse_dict)
    while any(bool(expr.free_symbols & cse_symbols) for expr in cse_dict.values()):
        cse_dict = {
            symbol: expr.xreplace(cse_dict) for symbol, expr in cse_dict.items()
        }
    cse_dict = {
        symbol: expr
        for symbol, expr in cse_dict.items()
        if symbol in cached_expression.free_symbols
    }
    return cached_expression, cse_dict

In [ ]:
cached_expression, cse = cache_constant_sub_expressions(
    optimized_expression, free_parameters
)
sp.count_ops(cached_expression)

In [ ]:
dot = sp.dotprint(expression.args[0].args[0].args[0].args[0], size=10)
graphviz.Source(dot)

In [ ]:
dot = sp.dotprint(optimized_expression.args[0].args[0].args[0].args[0], size=10)
graphviz.Source(dot)

In [ ]:
dot = sp.dotprint(cached_expression.args[0].args[0].args[0].args[0], size=10)
graphviz.Source(dot)

In [ ]:
for symbol, expr in cse.items():
    display(Math(sp.multiline_latex(symbol, expr, environment="eqnarray")))

In [ ]:
cache_converter = SympyDataTransformer.from_sympy(cse, backend="jax")
cached_data = cache_converter(data)
cached_phsp = cache_converter(phsp)

Further optimizations, such as identifying [common sub-expressions](https://github.com/ComPWA/tensorwaves/pull/374), are performed when the expression is expressed as a function in the computational backend. The effect of all optimization is illustrated below.

In [ ]:
# Express expressions in backend
function = create_parametrized_function(expression, parameter_defaults, backend="jax")
optimized_function = create_parametrized_function(
    optimized_expression, parameter_defaults, backend="jax"
)
cached_function = create_parametrized_function(
    cached_expression, parameter_defaults, backend="jax"
)
# Time performance on entire data set
%timeit -n100 function(data)
%timeit -n100 optimized_function(data_real)
%timeit -n100 cached_function(cached_data)

In [ ]:
optimizer = Minuit2(callback=CSVSummary("fit_traceback.csv"))

In [ ]:
estimator = UnbinnedNLL(function, data, phsp, backend="jax")
fit_result = optimizer.optimize(estimator, initial_parameters)
fit_result.execution_time

In [ ]:
estimator = UnbinnedNLL(optimized_function, data_real, phsp_real, backend="jax")
fit_result = optimizer.optimize(estimator, initial_parameters)
fit_result.execution_time

_Time estimate:_<br>
For this (deterministic) data sample and these initial parameter values, the fit requires **515 iterations** (GPU; 516 on CPU). On Google Colab, this should take **around 15 seconds** with GPU.

In [ ]:
estimator = UnbinnedNLL(cached_function, cached_data, cached_phsp, backend="jax")
fit_result = optimizer.optimize(estimator, initial_parameters)
fit_result.execution_time

In [ ]:
optimized_function.update_parameters(fit_result.parameter_values)
compare_model("m_12", data, phsp, optimized_function, bins=200)

## Visualize fit traceback

In [ ]:
traceback = np.genfromtxt("fit_traceback.csv", delimiter=",", names=True)
fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    figsize=(7, 8),
    gridspec_kw={"height_ratios": [1, 2.5]},
    sharex=True,
)
ax1.set_title("Negative log likelihood")
ax2.set_title("Parameter values")
ax2.set_xlabel("function call")
fig.tight_layout()

x = traceback["function_call"]
ax1.plot(x, traceback["estimator_value"])
for par in initial_parameters:
    label = f"${sp.latex(sp.Symbol(par))}$"
    key = re.sub(r"[\(\)]", "", par)
    ax2.plot(x, traceback[key], label=label)

legend = ax2.legend(loc=(0.77, 0.43))
legend.get_frame().set_alpha(None)
for par, line in zip(initial_parameters, ax2.get_lines()):
    label = line.get_label()
    color = line.get_color()
    ax2.axhline(
        y=original_parameters[par],
        color=color,
        alpha=0.5,
        linestyle="dotted",
    )